In [9]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.pyplot as plt
from pandas.plotting import table

pd.set_option("display.max_columns", None)

In [2]:
df = pd.read_csv('../data/default-of-credit-card-clients.csv')

### General Overview

In [6]:
df.shape

(30000, 25)

In [3]:
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,0,2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,0,0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [10]:
df.info()

# > all numerical attributes
# > no null values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   ID                          30000 non-null  int64
 1   LIMIT_BAL                   30000 non-null  int64
 2   SEX                         30000 non-null  int64
 3   EDUCATION                   30000 non-null  int64
 4   MARRIAGE                    30000 non-null  int64
 5   AGE                         30000 non-null  int64
 6   PAY_0                       30000 non-null  int64
 7   PAY_2                       30000 non-null  int64
 8   PAY_3                       30000 non-null  int64
 9   PAY_4                       30000 non-null  int64
 10  PAY_5                       30000 non-null  int64
 11  PAY_6                       30000 non-null  int64
 12  BILL_AMT1                   30000 non-null  int64
 13  BILL_AMT2                   30000 non-null  int64
 14  BILL_A

In [13]:
print(desc.to_latex())

\begin{tabular}{lrrrrrrrrrrrrrr}
\toprule
{} &   LIMIT\_BAL &       AGE &  BILL\_AMT1 &  BILL\_AMT2 &   BILL\_AMT3 &  BILL\_AMT4 &  BILL\_AMT5 &  BILL\_AMT6 &   PAY\_AMT1 &    PAY\_AMT2 &   PAY\_AMT3 &   PAY\_AMT4 &   PAY\_AMT5 &   PAY\_AMT6 \\
\midrule
count &    30000.00 &  30000.00 &   30000.00 &   30000.00 &    30000.00 &   30000.00 &   30000.00 &   30000.00 &   30000.00 &    30000.00 &   30000.00 &   30000.00 &   30000.00 &   30000.00 \\
mean  &   167484.32 &     35.49 &   51223.33 &   49179.08 &    47013.15 &   43262.95 &   40311.40 &   38871.76 &    5663.58 &     5921.16 &    5225.68 &    4826.08 &    4799.39 &    5215.50 \\
std   &   129747.66 &      9.22 &   73635.86 &   71173.77 &    69349.39 &   64332.86 &   60797.16 &   59554.11 &   16563.28 &    23040.87 &   17606.96 &   15666.16 &   15278.31 &   17777.47 \\
min   &    10000.00 &     21.00 & -165580.00 &  -69777.00 &  -157264.00 & -170000.00 &  -81334.00 & -339603.00 &       0.00 &        0.00 &       0.00 &       0.00 &  

In [14]:
#  Gender (1 = male; 2 = female).
#  Education (1 = graduate school; 2 = university; 3 = high school; 4 = others). 
#  Marital status (1 = married; 2 = single; 3 = others). 
#  default payment next month 	(1 = yes; 0 = no)

no_categorical = ['LIMIT_BAL','AGE','BILL_AMT1','BILL_AMT2','BILL_AMT3','BILL_AMT4','BILL_AMT5','BILL_AMT6','PAY_AMT1','PAY_AMT2','PAY_AMT3','PAY_AMT4','PAY_AMT5','PAY_AMT6']

desc = df[no_categorical].describe().round(2)

desc

,LIMIT_BAL,AGE,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
count,30000.00,30000.00,30000.00,30000.00,30000.00,30000.00,30000.00,30000.00,30000.00,30000.00,30000.00,30000.00,30000.00,30000.00
mean,167484.32,35.49,51223.33,49179.08,47013.15,43262.95,40311.40,38871.76,5663.58,5921.16,5225.68,4826.08,4799.39,5215.50
std,129747.66,9.22,73635.86,71173.77,69349.39,64332.86,60797.16,59554.11,16563.28,23040.87,17606.96,15666.16,15278.31,17777.47
min,10000.00,21.00,-165580.00,-69777.00,-157264.00,-170000.00,-81334.00,-339603.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,50000.00,28.00,3558.75,2984.75,2666.25,2326.75,1763.00,1256.00,1000.00,833.00,390.00,296.00,252.50,117.75
50%,140000.00,34.00,22381.50,21200.00,20088.50,19052.00,18104.50,17071.00,2100.00,2009.00,1800.00,1500.00,1500.00,1500.00
75%,240000.00,41.00,67091.00,64006.25,60164.75,54506.00,50190.50,49198.25,5006.00,5000.00,4505.00,4013.25,4031.50,4000.00
max,1000000.00,79.00,964511.00,983931.00,1664089.00,891586.00,927171.00,961664.00,873552.00,1684259.00,896040.00,621000.00,426529.00,528666.00


['ID',
 'PAY_AMT2',
 'PAY_AMT6',
 'BILL_AMT5',
 'PAY_AMT5',
 'PAY_AMT3',
 'PAY_AMT1',
 'BILL_AMT4',
 'BILL_AMT6',
 'BILL_AMT3',
 'BILL_AMT2',
 'BILL_AMT1',
 'LIMIT_BAL',
 'PAY_AMT4',
 'AGE']